In [1]:
import os

In [2]:
%pwd

'/mnt/e/bgremoveapp/bg_removal/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/mnt/e/bgremoveapp/bg_removal'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from bgremove.constants import *
from bgremove.utils.common import read_yaml, create_directories

In [7]:
print(CONFIG_FILE_PATH)

config/config.yaml


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
    
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir, 
            source_URL = config.source_URL, 
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir)
        return data_ingestion_config
    

In [9]:
# data Ingestion now

In [10]:
import os
import urllib.request as request
import zipfile
from bgremove import logger
from bgremove.utils.common import get_size
import gdown

In [11]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
#             file_name, headers = request.urlretrieve(
#                 url = self.config.source_URL,
#                 filename = self.config.local_data_file
#             )
#             url = 'https://drive.google.com/u/0/uc?id=1LqUU7BZeiq8I3i5KxApdOJ2haXm-cEv1&export=download'
            gdown.download(self.config.source_URL, self.config.local_data_file, quiet=False)
            file_name= self.config.local_data_file    
            logger.info(f"{file_name}Data has downlaod! with following info:\n")
        else:
            logger.info(f"File already exist of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        '''
            zip file extraction
            zip file path:str
            Extracts the zip file into the data directory
            Function return None
        '''
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2023-08-29 10:43:25,420: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-29 10:43:25,438: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-29 10:43:25,450: INFO: common: created directory at: artifacts]
[2023-08-29 10:43:25,471: INFO: common: created directory at: artifacts/data_ingestion]


Downloading...
From (uriginal): https://drive.google.com/u/0/uc?id=1LqUU7BZeiq8I3i5KxApdOJ2haXm-cEv1&export=download
From (redirected): https://drive.google.com/uc?id=1LqUU7BZeiq8I3i5KxApdOJ2haXm-cEv1&export=download&confirm=t&uuid=2d680721-8c01-4892-b146-c9d588215fa0
To: /mnt/e/bgremoveapp/bg_removal/artifacts/data_ingestion/data.zip
 12%|████████▌                                                            | 722M/5.79G [06:10<19:19, 4.37MB/s]

KeyboardInterrupt: 